# Logistic Reegression
1) import library sklean, pandas

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import random as rd
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn import preprocessing
import numpy as np
from sklearn.linear_model.logistic import  LogisticRegression
from sklearn import metrics
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import cross_val_score


2) Input data

In [8]:
X = pd.read_csv('../data_preprocessing/train_dummy.csv')
y=pd.read_csv('../data_preprocessing/training.csv')['IsBadBuy']

3) Split data into training and test data

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=33)

4) Train logistic model and caculate the accuracy of training data and test data (with bias)

In [30]:
model = LogisticRegression(fit_intercept = False, C = 1e9)
mdl = model.fit(X_train, y_train)
score_train = mdl.score(X_train,y_train)
score_test = mdl.score(X_test,y_test)
print("training data accuracy =",score_train,"; test data accuracy =",score_test)
roc_test = roc_auc_score(y_test, model.predict(X_test),average='weighted')
print("training data roc =",roc_test)

training data accuracy = 0.877994832446 ; test data accuracy = 0.874720255766
training data roc = 0.5


The accuracy is very high but the roc is not very good

5) get of bad and good samples

In [27]:
y_train_reset = y_train.reset_index(drop=True)
X_train_reset = X_train.reset_index(drop=True)
y_good_index = []
y_bad_index = []
index_new=[]
for i in range(0,len(y_train_reset)):
    if y_train_reset[i] == 0:
        y_good_index.append(i)
    else:
        y_bad_index.append(i) 

6) Get a list of same number of bad and good samples 

  a) have repeated records; X_train_new and y_train_new are the new lists

In [47]:
index_new_all = []
for i in range(0,7):
    y_good_index_new = []
    y_bad_index_new = []
    index_new = []
    rd.shuffle(y_good_index)
    rd.shuffle(y_good_index)
    y_good_index_new = rd.sample(y_good_index, 1000)
    y_bad_index_new = rd.sample(y_bad_index, 1000)         
    index_new = y_good_index_new+y_bad_index_new
    rd.shuffle(index_new)
    index_new_all = index_new_all+index_new

y_train_new = y_train_reset[index_new_all]
X_train_new = X_train_reset.loc[index_new_all,:]




12466


In [38]:
#Not use penalty in logistic regression in repeated records
model = LogisticRegression()
%time 
model_nopenalty = model.fit(X_train_new, y_train_new)

# check the accuracy on the training set and test set
nop_score_train = model_nopenalty.score(X_train_new, y_train_new)
nop_score_test = model_nopenalty.score(X_test, y_test)
print (nop_score_train)
print (nop_score_test)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.81 µs
0.688142857143
0.715779858415


In [37]:
roc_test = roc_auc_score(y_test, model.predict(X_test),average='weighted')
print("training data roc =",roc_test)

training data roc = 0.680111908995


  b)  have no repeated records; X_train_new2 and y_train_new2 are the new lists

In [49]:
#have no repeated record
y_good_index_new2 = rd.sample(y_good_index, 6233)
y_bad_index_new2 = rd.sample(y_bad_index, 6233) 
index_new2 = y_good_index_new2+y_bad_index_new2
y_train_new2 = y_train_reset[index_new2]
X_train_new2 = X_train_reset.loc[index_new2,:]
print(len(y_train_new2))

#Not use penalty in logistic regression in records without repetition
model2 = LogisticRegression()
%time 
model_nopenalty2 = model2.fit(X_train_new2, y_train_new2)

# check the accuracy on the training set and test set
nop_score_train2 = model_nopenalty2.score(X_train_new2, y_train_new2)
nop_score_test2 = model_nopenalty2.score(X_test, y_test)
print (nop_score_train2)
print (nop_score_test2)

CPU times: user 1e+03 ns, sys: 1e+03 ns, total: 2 µs
Wall time: 4.05 µs
0.683940317664
0.717515414478


Compared with the repeated records, the accuracy is close

7) logistic regression with penalty

In [52]:
##logistical regression
coef_l1_LR_all = []
coef_l2_LR_all = []
sparsity_l1_LR_all = []
sparsity_l2_LR_all = []
score_l1_LR_all = []
score_l2_LR_all = []
score_l1_LR_train_all = []
score_l2_LR_train_all = []

C_all = np.linspace(0.001,0.3, num=30)
for i, C in enumerate(C_all):
    clf_l1_LR = LogisticRegression(C=C, penalty='l1', tol=0.01)
    clf_l2_LR = LogisticRegression(C=C, penalty='l2', tol=0.01)
    clf_l1_LR.fit(X_train_new2, y_train_new2)
    clf_l2_LR.fit(X_train_new2, y_train_new2)
    
    coef_l1_LR = clf_l1_LR.coef_.ravel()
    coef_l2_LR = clf_l2_LR.coef_.ravel()
    coef_l1_LR_all.append(coef_l1_LR)
    coef_l2_LR_all.append(coef_l2_LR)
    
    sparsity_l1_LR = np.mean(coef_l1_LR == 0) * 100
    sparsity_l2_LR = np.mean(coef_l2_LR == 0) * 100
    score_l1_LR_train = clf_l1_LR.score(X_train_new2,y_train_new2)
    score_l2_LR_train = clf_l2_LR.score(X_train_new2,y_train_new2)
    score_l1_LR = clf_l1_LR.score(X_test,y_test)
    score_l2_LR = clf_l2_LR.score(X_test,y_test)
    roc_l1_LR = roc_auc_score(y_test, clf_l1_LR.predict(X_test))

    sparsity_l1_LR_all.append(sparsity_l1_LR)
    sparsity_l2_LR_all.append(sparsity_l2_LR)
    score_l1_LR_all.append(score_l1_LR)
    score_l2_LR_all.append(score_l2_LR)
    score_l1_LR_train_all.append(score_l1_LR_train)
    score_l2_LR_train_all.append(score_l2_LR_train)
    
    print("C=%.2f" % C)
    print("Sparsity with L1 penalty: %.2f%%" % sparsity_l1_LR)
    print("training data score with L1 penalty: %.4f" % score_l1_LR_train)
    print("score with L1 penalty: %.4f" % score_l1_LR)
    print("test data roc score with L1 penalty: %.4f" % roc_l1_LR)

    print("Sparsity with L2 penalty: %.2f%%" % sparsity_l2_LR)
    print("score with L2 penalty: %.4f" % score_l2_LR)
    print("training data score with L2 penalty: %.4f" % score_l2_LR_train)

plt.figure(1)
plt.subplot(211)
plt.plot(C_all[1:30], sparsity_l1_LR_all[1:30], 'k')
plt.plot(C_all[1:30], sparsity_l2_LR_all[1:30], 'r--')
plt.grid(True)
plt.figure(1)
plt.subplot(212)
plt.plot(C_all[1:30], score_l1_LR_train_all[1:30],'k')
#plt.plot(C_all[1:30], score_l1_LR_train_all[1:30],'b')
plt.grid(True)



C=0.00
Sparsity with L1 penalty: 90.97%
training data score with L1 penalty: 0.6053
score with L1 penalty: 0.5791
test data roc score with L1 penalty: 0.6028
Sparsity with L2 penalty: 4.86%
score with L2 penalty: 0.5675
training data score with L2 penalty: 0.6009
C=0.01
Sparsity with L1 penalty: 87.50%
training data score with L1 penalty: 0.6766
score with L1 penalty: 0.7366
test data roc score with L1 penalty: 0.6736
Sparsity with L2 penalty: 4.86%
score with L2 penalty: 0.5682
training data score with L2 penalty: 0.6011
C=0.02
Sparsity with L1 penalty: 79.17%
training data score with L1 penalty: 0.6838
score with L1 penalty: 0.7435
test data roc score with L1 penalty: 0.6802
Sparsity with L2 penalty: 4.86%
score with L2 penalty: 0.5675
training data score with L2 penalty: 0.6009
C=0.03
Sparsity with L1 penalty: 78.47%
training data score with L1 penalty: 0.6870
score with L1 penalty: 0.7463
test data roc score with L1 penalty: 0.6849
Sparsity with L2 penalty: 4.86%
score with L2 pena

In [53]:
print(score_l1_LR_all)
# from the score_l1_LR_all we can see the fifth value is highest

[0.5791276547156885, 0.73656085864352594, 0.74345741036766388, 0.74633477963005257, 0.74770495546928517, 0.74756793788536191, 0.74569536423841054, 0.74638045215802695, 0.74537565654258964, 0.74496460379081986, 0.7451472939027175, 0.74505594884676862, 0.74487325873487098, 0.74368577300753602, 0.74437086092715232, 0.74414249828728019, 0.74350308289563827, 0.74382279059145928, 0.74400548070335693, 0.74290934003197073, 0.74382279059145928, 0.74309203014386849, 0.74309203014386849, 0.74217857958438005, 0.7419958894724823, 0.74158483672071251, 0.74053436857730071, 0.7419958894724823, 0.74062571363324958, 0.74195021694450791]


In [56]:
#the best one the fifth time fitting
list = np.where(coef_l1_LR_all[4] != 0)[0].tolist()
head = X.columns.values.tolist()
best_test_score = score_l1_LR_all[4]
best_train_score = score_l1_LR_train_all[4]
print(best_test_score)
#head(list)
need_head = []
for i in list:
    need_head.append(head[i])
print(need_head)

0.747704955469
['VehicleAge', 'VehOdo', 'MMRAcquisitionAuctionAveragePrice', 'MMRAcquisitionAuctionCleanPrice', 'MMRAcquisitionRetailAveragePrice', 'MMRAcquisitonRetailCleanPrice', 'MMRCurrentAuctionAveragePrice', 'MMRCurrentAuctionCleanPrice', 'MMRCurrentRetailAveragePrice', 'MMRCurrentRetailCleanPrice', 'BYRNO', 'VehBCost', 'WarrantyCost', 'Auction_ADESA', 'Auction_MANHEIM', 'Make_CHEVROLET', 'Make_DODGE', 'Make_FORD', 'Color_BLUE', 'Color_RED', 'WheelTypeID_1.0', 'WheelTypeID_2.0', 'WheelTypeID_U0', 'Nationality_AMERICAN', 'Size_LARGE', 'Size_LARGE SUV', 'Size_MEDIUM', 'Size_MEDIUM SUV', 'Size_VAN', 'TopThreeAmericanName_GM', 'PRIMEUNIT_NO', 'AUCGUART_GREEN', 'VNST_FL', 'VNST_NC', 'VNST_TX']
